In [35]:
from frgpascal.analysis.processing import load_all, compress_jv, get_worklist_times
from frgpascal.analysis.brightfield import load_image
from frgtrainview.analysis.crop import crop_pl
from scipy import stats
from natsort import natsorted
from natsort import index_natsorted
import copy
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
import seaborn as sns
import warnings
from tqdm import tqdm
from matplotlib import style
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
import os
import json
import pickle as pkl
import time
from PIL import Image

# plotting settings
# %config InlineBackend.figure_format = 'retina'
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["axes.linewidth"] = 1.75  # set the value globally

import os
import glob
import shutil

In [2]:
source = '/Users/deniz/Documents/GitHub/PASCAL_TrainView/analysis_experiments/rotate_CNN/raw_data'
destination = f'/Users/deniz/Documents/GitHub/PASCAL_TrainView/analysis_experiments/rotate_CNN/PL_images'

In [3]:
# move just the PL images and add batch id to the file name
for folder, subfolders, files in os.walk(source):
    if folder != source:
        for f in files:
            if f.endswith("5000ms.tif"):
                # print("File Name: ", f)
                # print(f"Path: ", os.path.join(folder, f))
                file_name_temp = f
                file_path_temp = os.path.join(folder, f)
                file_path_dir = os.path.dirname(file_path_temp)
                batch_name = file_path_dir.split('_')[5].split('-')[0]
                os.rename(f'{file_path_temp}', f'{file_path_dir}/{batch_name}_{file_name_temp}')
                renamed_path_temp = os.path.join(file_path_dir, f'{batch_name}_{file_name_temp}')
                shutil.move(renamed_path_temp, destination)

In [4]:
crop_desitnation = '/Users/deniz/Documents/GitHub/PASCAL_TrainView/analysis_experiments/rotate_CNN/cropped_PL_images'
source = '/Users/deniz/Documents/GitHub/PASCAL_TrainView/analysis_experiments/rotate_CNN/raw_PL_images'

In [38]:
fids[0]

'/Users/deniz/Documents/GitHub/PASCAL_TrainView/analysis_experiments/rotate_CNN/raw_PL_images/B14_sample22_plimage_5000ms.tif'

In [39]:
os.path.basename(fids[0])

'B14_sample22_plimage_5000ms.tif'

In [5]:
# crop the PL images
fids = glob.glob(f'{source}/*.tif')


In [43]:
for fid in fids:
    base_name = os.path.basename(fid)
    temp_image = load_image(fid)
    crop_image = crop_pl(np.uint8(temp_image * 255), output_shape=(256,256))[0]
    Image.fromarray(crop_image).save(f'{crop_desitnation}/{base_name}')